<a href="https://colab.research.google.com/github/ddekun/Recommendation_systems/blob/lesson2/lesson2/hw_webinar_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Рекомендательные системы

## Урок 2. Бейзлайны и детерминированные алгоритмы item-item

### Домашнее задание

In [6]:
!pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
# !pip install jupyter_server

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k
#from tqdm.notebook import trange, tqdm
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

/usr/local/lib/python3.9/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
data = pd.read_csv('/content/drive/MyDrive/Geekbrains/Recommendation systems/lesson2/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [11]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

**1. Бейзлайны**

Создадим датафрейм с покупками юзеров на тестовом датасете (последние 3 недели)

In [12]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [13]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [14]:
def item_weights(df):
    sales_sum = df.groupby('item_id')['sales_value'].sum().reset_index()
    sales_sum = sales_sum[sales_sum['sales_value'] > 1]
    sales_sum['weight'] = np.log(sales_sum['sales_value'] + 0.000001)
    weight_sum = sales_sum['weight'].sum()
    sales_sum['weight'] = sales_sum['weight'] / weight_sum
    sales_sum.drop('sales_value', axis=1, inplace=True)
    return sales_sum

In [15]:
df_w = item_weights(data_train)
pd.set_option('display.float_format', '{:.6f}'.format)
df_w

,item_id,weight
0,25671,0.000013
2,26093,0.000002
3,26190,0.000002
4,26355,0.000003
5,26426,0.000004
...,...,...
86859,17330511,0.000010
86861,17382205,0.000009
86862,17383227,0.000007
86863,17827644,0.000004


In [16]:
round(df_w['weight'].sum(),10)

1.0

In [17]:
df_n = df_w.to_numpy()

In [18]:
def weighted_random_recommendation(items_weights, n=3):
    """Случайные рекоммендации
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    items = items_weights[:,0]
    recs = np.random.choice(items, size=n, replace=False, p=items_weights[:,1])
    
    return [int(x) for x in recs.tolist()]

In [19]:
%time
result['wr_recommend'] = result['user_id'].progress_apply(lambda x: weighted_random_recommendation(df_n, n=5))

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


  0%|          | 0/2042 [00:00<?, ?it/s]

In [20]:
result.head(2)

,user_id,actual,wr_recommend
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[867607, 12386154, 833159, 989249, 9855072]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1060163, 990289, 1043506, 819423, 15973034]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [21]:
df_pred = pd.read_csv('/content/drive/MyDrive/Geekbrains/Recommendation systems/lesson2/predictions_basic.csv')
df_pred.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[903554, 952254, 8205795, 929248, 12781871]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1006342, 869622, 960942, 940108, 949023, 819840]","[948298, 819840, 869622, 999090, 1002459, 949023]","[820895, 819304, 545926, 480014, 819594, 820165]","[949023, 976998]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[15597081, 9555168, 13213464, 7442759, 1046689]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1006342, 869622, 960942, 940108, 949023, 819840]","[948298, 819840, 869622, 999090, 1002459, 949023]","[820895, 819304, 545926, 480014, 819594, 820165]","[949023, 976998]"


In [22]:
# your_code
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2042 entries, 0 to 2041
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   user_id                 2042 non-null   int64 
 1   actual                  2042 non-null   object
 2   random_recommendation   2042 non-null   object
 3   popular_recommendation  2042 non-null   object
 4   itemitem                2042 non-null   object
 5   cosine                  2042 non-null   object
 6   tfidf                   2042 non-null   object
 7   own_purchases           2042 non-null   object
dtypes: int64(1), object(7)
memory usage: 127.8+ KB


In [23]:
df_pred.actual[0]

'[  821867   834484   856942   865456   889248   907957   914190   943316\n   951954   954486   958046   962568   969231   971585   979707   986947\n   990656   995242  1004906  1005186  1042083  1050310  1060819  1062002\n  1064441  1069733  1074612  1082185  1131115  1132771  6534544 13876341\n 15971874 17178953   883616   917704   931860   961554  1002032  1031190\n  8090541  8293439  9297615  9527329 15926712  1049998   861272   869465\n   877373   908213   933913   940947   945809   959316   978974  1031697\n  1041796  1048918  1081189  1101422  1115576  1122428  1132231  1132814\n  5577022  8091601  9296986  9677939 10356149 13417048 15741823 15830875]'

In [24]:
df_pred.random_recommendation[0]

'[903554, 952254, 8205795, 929248, 12781871]'

In [25]:
def str_to_int(row):
    list_int = []
    for i in row.strip('[]').split():
        list_int.append(int(i.rstrip(',')))
    return list_int

In [26]:
str_to_int(df_pred.random_recommendation[0])

[903554, 952254, 8205795, 929248, 12781871]

In [27]:
for col in df_pred.select_dtypes(include='object').columns:
    df_pred[col] = df_pred.apply(lambda x: str_to_int(x[col]), axis=1)

In [28]:
%time
df_pred['wr_recommend'] = df_pred['user_id'].progress_apply(lambda x: weighted_random_recommendation(df_n, n=5))

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.87 µs


  0%|          | 0/2042 [00:00<?, ?it/s]

In [29]:
df_pred.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,wr_recommend
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[903554, 952254, 8205795, 929248, 12781871]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1006342, 869622, 960942, 940108, 949023, 819840]","[948298, 819840, 869622, 999090, 1002459, 949023]","[820895, 819304, 545926, 480014, 819594, 820165]","[949023, 976998]","[7429099, 1119644, 1021461, 983000, 6396174]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[15597081, 9555168, 13213464, 7442759, 1046689]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1006342, 869622, 960942, 940108, 949023, 819840]","[948298, 819840, 869622, 999090, 1002459, 949023]","[820895, 819304, 545926, 480014, 819594, 820165]","[949023, 976998]","[7169144, 9527051, 934663, 114642, 5581553]"


In [30]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)

    return precision

In [31]:
for col in df_pred.iloc[:,2:]:
    print(f'{col.title():22s} : {df_pred.apply(lambda row: precision_at_k(row[col], row["actual"]), axis=1).mean():.5f}')

Random_Recommendation  : 0.00020
Popular_Recommendation : 0.15524
Itemitem               : 0.00666
Cosine                 : 0.00470
Tfidf                  : 0.00196
Own_Purchases          : 0.01934
Wr_Recommend           : 0.00098


Наилучший результат дал метод "Популярные рекомендации" (Popular_Recommendation)

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [27]:
# your_code

In [32]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [33]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [34]:
df_pred2 = df_pred.copy()

### Улучшение случайных рекомендаций

In [35]:
def random_recommend_top(items, n=5):
    """Случайные рекоммендации"""
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [36]:
%%time
items = top_5000
df_pred2['random_recommendation'] = df_pred2['user_id'].apply(lambda x: random_recommend_top(items))

CPU times: user 2.08 s, sys: 16.9 ms, total: 2.1 s
Wall time: 7.71 s


In [37]:
import warnings
warnings.filterwarnings('ignore')

### Улучшение Item-Item Recommender

In [38]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [39]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [ ]:
%%time
model = ItemItemRecommender(K=3, num_threads=8)                          # K - кол-во ближайших соседей
model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)    # На вход item-user matrix
recs = model.recommend(userid=userid_to_id[2],                           # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(), # на вход user-item matrix
                        N=5,                                             # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

In [ ]:
%%time
df_pred2['itemitem'] = df_pred2['user_id'].\
    map(lambda x: [id_to_itemid[rec] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)[0]])

### Улучшение Weights_top Recommender

In [40]:
def item_weights_top(df, top=5000):
    sales_sum = df.groupby('item_id')['sales_value'].sum().reset_index()
    sales_sum = sales_sum[sales_sum['sales_value'] > 1]
    sales_sum.sort_values('sales_value', ascending=False, inplace=True)
    sales_sum = sales_sum.head(top)
        
    sales_sum['weight'] = np.log(sales_sum['sales_value'] + 0.000001)
    weight_sum = sales_sum['weight'].sum()
    sales_sum['weight'] = sales_sum['weight'] / weight_sum
    sales_sum.drop('sales_value', axis=1, inplace=True)
    return sales_sum

In [41]:
df_w2 = item_weights_top(data_train)
df_w2

,item_id,weight
2381,999999,0.000497
4346,6534178,0.000434
4339,6533889,0.000354
2757,1029743,0.000350
4345,6534166,0.000344
...,...,...
4977,14043825,0.000096
4979,14050436,0.000096
4978,14043826,0.000094
4975,14043817,0.000093


In [42]:
round(df_w2['weight'].sum(),10)

1.0

In [43]:
df_n2 = df_w2.to_numpy()

In [44]:
%time
df_pred2['wr_recommend'] = df_pred2['user_id'].progress_apply(lambda x: weighted_random_recommendation(df_n2, n=5))

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.82 µs


  0%|          | 0/2042 [00:00<?, ?it/s]

In [45]:
for col in df_pred2.iloc[:,2:]:
    print(f'{col.title():22s} : {df_pred2.apply(lambda row: precision_at_k(row[col], row["actual"]), axis=1).mean():.5f}')

Random_Recommendation  : 0.00441
Popular_Recommendation : 0.15524
Itemitem               : 0.00666
Cosine                 : 0.00470
Tfidf                  : 0.00196
Own_Purchases          : 0.01934
Wr_Recommend           : 0.00686
